In [1]:
import pandas as pd

In [2]:
url = 'https://github.com/mattharrison/datasets/raw/master/data/dirtydevil.txt'

In [3]:
df = pd.read_csv(url, skiprows=lambda num: num < 34 or num == 35, sep='\t')

/tmp/ipykernel_43/879812612.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url, skiprows=lambda num: num < 34 or num == 35, sep='\t')


In [4]:
def to_denver_time(df_, time_col, tz_col):
    return(df_
          .assign(**{tz_col: df_[tz_col].replace('MDT', 'MST7MDT')})
          .groupby(tz_col)
          [time_col]
          .transform(lambda s: pd.to_datetime(s)
                    .dt.tz_localize(s.name, ambiguous=True)
                    .dt.tz_convert('America/Denver'))
          )

In [5]:
def tweak_river(df_):
    return (df_
           .assign(datetime=to_denver_time(df_, 'datetime', 'tz_cd'))
           .rename(columns={'144166_00060': 'cfs',
                           '144167_00065': 'gage_height'})
           .set_index('datetime')
           )

In [6]:
dd = tweak_river(df)

In [7]:
dd

,agency_cd,site_no,tz_cd,cfs,144166_00060_cd,gage_height,144167_00065_cd
datetime,,,,,,,
2001-05-07 01:00:00-06:00,USGS,9333500,MDT,71.00,A:[91],NaN,NaN
2001-05-07 01:15:00-06:00,USGS,9333500,MDT,71.00,A:[91],NaN,NaN
2001-05-07 01:30:00-06:00,USGS,9333500,MDT,71.00,A:[91],NaN,NaN
2001-05-07 01:45:00-06:00,USGS,9333500,MDT,70.00,A:[91],NaN,NaN
2001-05-07 02:00:00-06:00,USGS,9333500,MDT,70.00,A:[91],NaN,NaN
...,...,...,...,...,...,...,...
2020-09-28 08:30:00-06:00,USGS,9333500,MDT,9.53,P,6.16,P
2020-09-28 08:45:00-06:00,USGS,9333500,MDT,9.20,P,6.15,P
2020-09-28 09:00:00-06:00,USGS,9333500,MDT,9.20,P,6.15,P


In [8]:
print(dd.head(5).to_csv())

datetime,agency_cd,site_no,tz_cd,cfs,144166_00060_cd,gage_height,144167_00065_cd
2001-05-07 01:00:00-06:00,USGS,9333500,MDT,71.0,A:[91],,
2001-05-07 01:15:00-06:00,USGS,9333500,MDT,71.0,A:[91],,
2001-05-07 01:30:00-06:00,USGS,9333500,MDT,71.0,A:[91],,
2001-05-07 01:45:00-06:00,USGS,9333500,MDT,70.0,A:[91],,
2001-05-07 02:00:00-06:00,USGS,9333500,MDT,70.0,A:[91],,



In [9]:
dd.to_excel('/tmp/dd.xlsx')

ValueError: Excel does not support datetimes with timezones. Please ensure that datetimes are timezone unaware before writing to Excel.

In [ ]:
(dd
    .reset_index()
    .assign(datetime=lambda df_: df_.datetime.dt.tz_convert(tz=None))
    .set_index('datetime')
    .to_excel('/tmp/dd.xlsx')
)

In [12]:
writer = pd.ExcelWriter('/tmp/dd2.xlsx')

In [13]:
dd2 = (dd
      .reset_index()
      .assign(datetime=lambda df_: df_.datetime.dt.tz_convert(tz=None))
      .set_index('datetime')
      )

In [14]:
(dd2
    .loc['2010':'2010-12-31']
    .to_excel(writer, sheet_name='2010')
)

In [15]:
(dd2
    .loc['2011':'2011-12-31']
    .to_excel(writer, sheet_name='2011')
)

In [16]:
writer.save()

AttributeError: 'OpenpyxlWriter' object has no attribute 'save'

In [17]:
dd.to_feather('/tmp/dd.fea')

ValueError: feather does not support serializing <class 'pandas.core.indexes.datetimes.DatetimeIndex'> for the index; you can .reset_index() to make the index into column(s)

In [18]:
(dd
    .reset_index()
    .to_feather('/tmp/dd.fea')
)

In [19]:
dd2 = pd.read_feather('/tmp/dd.fea')

In [20]:
dd2.set_index('datetime').equals(dd)

True

In [21]:
import sqlite3

In [22]:
con = sqlite3.connect('dd.db')

In [23]:
dd.to_sql('dd', con, if_exists='replace')

539305

In [24]:
import sqlalchemy as sa

In [25]:
eng = sa.create_engine('sqlite:///dd.db')

In [26]:
sa_con = eng.connect()

In [27]:
dd2 = pd.read_sql('dd', sa_con, index_col='datetime')

In [28]:
dd2.equals(dd)

False

In [29]:
dd2

,agency_cd,site_no,tz_cd,cfs,144166_00060_cd,gage_height,144167_00065_cd
datetime,,,,,,,
2001-05-07 01:00:00-06:00,USGS,9333500,MDT,71.00,A:[91],NaN,None
2001-05-07 01:15:00-06:00,USGS,9333500,MDT,71.00,A:[91],NaN,None
2001-05-07 01:30:00-06:00,USGS,9333500,MDT,71.00,A:[91],NaN,None
2001-05-07 01:45:00-06:00,USGS,9333500,MDT,70.00,A:[91],NaN,None
2001-05-07 02:00:00-06:00,USGS,9333500,MDT,70.00,A:[91],NaN,None
...,...,...,...,...,...,...,...
2020-09-28 08:30:00-06:00,USGS,9333500,MDT,9.53,P,6.16,P
2020-09-28 08:45:00-06:00,USGS,9333500,MDT,9.20,P,6.15,P
2020-09-28 09:00:00-06:00,USGS,9333500,MDT,9.20,P,6.15,P


In [31]:
(dd2
    .reset_index()
    .assign(datetime=lambda df_: df_.datetime
           .dt.tz_localize('America/Denver', ambiguous=False))
    .set_index('datetime')
    .equals(dd)
)

TypeError: Already tz-aware, use tz_convert to convert.

In [32]:
obj = dd.to_dict()

In [35]:
dd2 = pd.DataFrame.from_dict(obj)

In [36]:
dd.equals(dd2)

True

In [37]:
dd.to_json('/tmp/dd.json.gz')

In [38]:
dd2 = pd.read_json('/tmp/dd.json')

FileNotFoundError: File /tmp/dd.json does not exist

In [39]:
dd2

,agency_cd,site_no,tz_cd,cfs,144166_00060_cd,gage_height,144167_00065_cd
2001-05-07 01:00:00-06:00,USGS,9333500,MDT,71.00,A:[91],NaN,NaN
2001-05-07 01:15:00-06:00,USGS,9333500,MDT,71.00,A:[91],NaN,NaN
2001-05-07 01:30:00-06:00,USGS,9333500,MDT,71.00,A:[91],NaN,NaN
2001-05-07 01:45:00-06:00,USGS,9333500,MDT,70.00,A:[91],NaN,NaN
2001-05-07 02:00:00-06:00,USGS,9333500,MDT,70.00,A:[91],NaN,NaN
...,...,...,...,...,...,...,...
2020-09-28 08:30:00-06:00,USGS,9333500,MDT,9.53,P,6.16,P
2020-09-28 08:45:00-06:00,USGS,9333500,MDT,9.20,P,6.15,P
2020-09-28 09:00:00-06:00,USGS,9333500,MDT,9.20,P,6.15,P
2020-09-28 09:15:00-06:00,USGS,9333500,MDT,9.20,P,6.15,P


In [40]:
dd2.equals(dd)

True

In [41]:
dd3 = (dd2
      .reset_index()
      .rename(columns={'index':'datetime'})
      .assign(datetime=lambda df_: df_.datetime.dt.tz_localize(tz='UTC')
             .dt.tz_convert('America/Denver'))
      .set_index('datetime')
      )

TypeError: Already tz-aware, use tz_convert to convert.

In [43]:
dd3

NameError: name 'dd3' is not defined

In [44]:
dd3.equals(dd)

NameError: name 'dd3' is not defined

In [45]:
dd3.round(3).equals(dd)

NameError: name 'dd3' is not defined